In [2]:
import numpy as np
from tqdm import tqdm
import os

In [5]:
def get_random_samples_by_label(base, n_samples, label):
    samples = base[base[:, -1] == label]

    if n_samples:
        indexes = np.random.choice(samples.shape[0], n_samples, replace=False)
        return samples[indexes]

    return samples


def get_random_samples(base, qtd_segments):
    # qtd_segments each label
    qtd_segments = int(qtd_segments / 2)
    negatives = get_random_samples_by_label(base=base, n_samples=qtd_segments, label=0)
    positives = get_random_samples_by_label(base=base, n_samples=qtd_segments, label=1)
    base_ready = np.vstack((positives, negatives))
    np.random.shuffle(base_ready)
    return base_ready


def extract_segments_bins(databases, m_bins, qtd_segments, segment_size, range_hist):
    directory = "./segments_bins"
    if not os.path.exists(directory):
        os.makedirs(directory)

    filename = f"mb{m_bins}_ss{segment_size}_rh{range_hist[0]}-{range_hist[1]}.npy"
    path = os.path.join(directory, filename)

    if os.path.isfile(path):
        segment_bins = np.load(path)
        return get_random_samples(segment_bins, qtd_segments)

    base = np.empty((0, m_bins + 1))

    for db in databases:
        label = 0 if db == "NSRDB" else 1

        rri_output = np.load(f"../extract_rri/rri/{db}.npy")

        rri_size = rri_output.shape[0]

        segments = np.empty((0, segment_size))
        print("arange in segments...")
        for i in tqdm(range(0, rri_size, segment_size)):
            segment = rri_output[i : i + segment_size]
            if segment.shape[0] == segment_size:
                segments = np.vstack((segments, segment))

        print("calculating histograms...")
        for segment in tqdm(segments):
            hist, _ = np.histogram(a=segment, range=range_hist, bins=m_bins)
            x = hist
            y = label
            row = np.append(x, y)
            row = row.reshape((1, row.shape[0]))
            base = np.vstack((base, row))

    num_rows = base.shape[0]
    permutation = np.random.permutation(num_rows)
    shuffled_base = base[permutation].copy()

    np.save(path, shuffled_base)

    return get_random_samples(shuffled_base, qtd_segments)

In [30]:
# base = extract_segments_bins(databases=DATABASES)